In [ ]:
import os, random, numpy as np, torch

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("Capability:", torch.cuda.get_device_capability(0))

PyTorch: 2.6.0+cu124
CUDA available: False


In [ ]:
!pip -q install "datasets>=2.20.0" "transformers>=4.42.0" "accelerate>=0.33.0" "evaluate>=0.4.2" "huggingface_hub>=0.23.0" gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, TrainingArguments, Trainer)
import evaluate, numpy as np, torch, re, os, json

# Mixed precision flags (handles fp16 vs bf16 automatically)
bf_major = torch.cuda.get_device_capability(0)[0] if torch.cuda.is_available() else 0
BF16 = torch.cuda.is_available() and bf_major >= 8  # A100/L4 etc.
FP16 = torch.cuda.is_available() and not BF16

print({"BF16": BF16, "FP16": FP16})


{'BF16': False, 'FP16': False}


In [ ]:
dataset = load_dataset("zou-lab/MedCaseReasoning")
dataset



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/897 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/99.3M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/7.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13092 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/897 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'pmcid', 'title', 'journal', 'article_link', 'publication_date', 'text', 'case_prompt', 'diagnostic_reasoning', 'final_diagnosis'],
        num_rows: 13092
    })
    val: Dataset({
        features: ['Unnamed: 0', 'pmcid', 'title', 'journal', 'article_link', 'publication_date', 'text', 'case_prompt', 'diagnostic_reasoning', 'final_diagnosis'],
        num_rows: 500
    })
    test: Dataset({
        features: ['Unnamed: 0', 'pmcid', 'title', 'journal', 'article_link', 'publication_date', 'text', 'case_prompt', 'diagnostic_reasoning', 'final_diagnosis'],
        num_rows: 897
    })
})

In [ ]:
sample = dataset["train"][0]
{ k: sample[k] for k in ["case_prompt", "diagnostic_reasoning", "final_diagnosis"] }


{'case_prompt': 'A 30-year-old man presented with several months of pain, tenderness, and swelling on the left side of his palate. On examination, the left palatine mucosa was diffusely swollen—most pronounced in the molar region—with poorly defined borders and increased pain on palpation. T2-weighted MRI demonstrated a high-signal-intensity lesion in that area. Panoramic radiography and CT scans of the maxilla and sinuses were unremarkable, and all left maxillary teeth appeared healthy. His medical and family histories were unremarkable, and he denied any history of trauma or inflammation in the head or neck. Empiric treatment with oral antibiotics and nonsteroidal anti-inflammatory drugs for 7 days produced no improvement. Based on the clinical and radiographic findings, a soft-tissue tumor was suspected, and an incisional biopsy was performed.',
 'diagnostic_reasoning': '1. Infection or inflammation was considered given the pain and swelling—“we administered antibiotics orally … and

In [ ]:
def is_valid(ex):
    return all([
        isinstance(ex.get("case_prompt", None), str) and len(ex["case_prompt"].strip())>0,
        isinstance(ex.get("diagnostic_reasoning", None), str) and len(ex["diagnostic_reasoning"].strip())>0,
        isinstance(ex.get("final_diagnosis", None), str) and len(ex["final_diagnosis"].strip())>0,
    ])

dataset = dataset.filter(is_valid)
for split in dataset:
    print(split, len(dataset[split]))


Filter:   0%|          | 0/13092 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/897 [00:00<?, ? examples/s]

train 13092
val 500
test 897


In [ ]:
MODEL_NAME = "t5-small"   # fast + decent; upgrade to "google/flan-t5-base" if you have time
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
# Keep outputs structured so you can parse later
def format_target(reasoning: str, diagnosis: str) -> str:
    reasoning = reasoning.strip()
    diagnosis = diagnosis.strip()
    return f"Reasoning: {reasoning} || Diagnosis: {diagnosis}"

def preprocess_function(batch):
    inputs  = [f"Symptoms: {x}" for x in batch["case_prompt"]]
    targets = [format_target(r, d) for r, d in zip(batch["diagnostic_reasoning"], batch["final_diagnosis"])]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(targets, max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

cols_to_remove = dataset["train"].column_names  # remove original cols after tokenize to save memory
tokenized = dataset.map(preprocess_function, batched=True, remove_columns=cols_to_remove)

tokenized


Map:   0%|          | 0/13092 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/897 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13092
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 897
    })
})

In [ ]:
train_ds = tokenized["train"]
eval_ds  = tokenized["val"]
test_ds  = tokenized["test"]

# SPEED MODE (uncomment for a quick demo run)
# train_ds = train_ds.select(range(2000))
# eval_ds  = eval_ds.select(range(300))
# test_ds  = test_ds.select(range(300))

len(train_ds), len(eval_ds), len(test_ds)


(13092, 500, 897)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
!pip -q install rouge-score

import evaluate
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    import numpy as np
    from transformers import AutoTokenizer
    # we already have tokenizer in the notebook; this is just to be safe if you re-run cells
    global tokenizer

    preds, labels = eval_pred
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 with pad_token_id then decode labels
    import numpy as np
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Compute ROUGE
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 2) for k, v in result.items()}


  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -U transformers


In [ ]:
from transformers import TrainingArguments
print(TrainingArguments.__module__)
help(TrainingArguments)


transformers.training_args
Help on class TrainingArguments in module transformers.training_args:

class TrainingArguments(builtins.object)
 |  TrainingArguments(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[tra

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./medprompt-t5-small",
    eval_strategy="epoch",           # <-- use eval_strategy (not evaluation_strategy)
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,              # bump to 2–3 if you have time
    weight_decay=0.01,
    warmup_ratio=0.1,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    fp16=True,                       # or bf16=True if your GPU supports it
    report_to=None,                  # or "none"
    seed=42,
)


In [ ]:
# after loading the model
model.config.max_length = 256


In [ ]:
# Patch: make sure TrainingArguments has generation_config
if not hasattr(training_args, "generation_config"):
    training_args.generation_config = None  # or a path/string if you actually want to load one

from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq, GenerationConfig

# keep your existing tokenizer/model/datasets
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# optional default generation settings used by trainer.predict()
model.generation_config = GenerationConfig(max_new_tokens=256, num_beams=4)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
    processing_class=tokenizer,  # avoids tokenizer deprecation warning
)

print("Trainer ready ✅")


Trainer ready ✅


In [ ]:
from transformers import GenerationConfig

# 1) Ensure tokenizer has pad/eos
if tokenizer.pad_token_id is None:
    # For T5 this should already be set, but just in case:
    tokenizer.pad_token = tokenizer.eos_token or tokenizer.convert_ids_to_tokens(0)

# 2) Fix model decoding config
if getattr(model.config, "decoder_start_token_id", None) is None:
    model.config.decoder_start_token_id = tokenizer.pad_token_id  # T5 convention

if getattr(model.config, "eos_token_id", None) is None:
    # T5 uses </s>
    model.config.eos_token_id = tokenizer.eos_token_id or tokenizer.pad_token_id

# 3) Optional: set a default generation config (used by trainer.predict / model.generate)
model.generation_config = GenerationConfig(
    max_new_tokens=256,
    num_beams=4,
    decoder_start_token_id=model.config.decoder_start_token_id,
    eos_token_id=model.config.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

# 4) If your Seq2SeqTrainer expects args fields, make sure they exist
setattr(training_args, "predict_with_generate", True)
setattr(training_args, "generation_max_length", None)  # leave None when using max_new_tokens
setattr(training_args, "generation_num_beams", 4)


In [ ]:
pred = trainer.predict(
    eval_ds,
    max_new_tokens=256,
    num_beams=4,
    decoder_start_token_id=model.config.decoder_start_token_id,
    eos_token_id=model.config.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ms3003 (ms3003-boston-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import numpy as np, evaluate
rouge = evaluate.load("rouge")

decoded_preds = tokenizer.batch_decode(pred.predictions, skip_special_tokens=True)
labels = pred.label_ids
labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

metrics = {k: round(v*100, 2) for k,v in rouge.compute(predictions=decoded_preds, references=decoded_labels).items()}
metrics


{'rouge1': np.float64(16.64),
 'rouge2': np.float64(2.8),
 'rougeL': np.float64(10.39),
 'rougeLsum': np.float64(10.38)}

In [ ]:
for i in range(3):
    print("PRED:", decoded_preds[i][:400], "\nREF :", decoded_labels[i][:400], "\n"+"-"*80)


PRED: a 48-year-old postmenopausal woman (gravida 2, para 2) presented with a 3-month history of brown-yellowish vaginal discharge, fluctuating lower abdominal pain, fever, and progressive abdominal enlargement over 15 years, worsening in the last 5 years. she reported difficulty with defecation and voiding. she underwent an attempted myomectomy in the past, but removal was aborted due to severe adhesio 
REF : Reasoning: 1. Consideration of intraligamentary myoma with cystic degeneration — “suspected to originate from intraligamentery myoma with cystic degeneration...” 2. Consideration of left ovarian fibroid — “differential diagnosis of left ovarian fibroid.” 3. Consideration of infected hematometra — “hypoechoic mass in the uterine cavity, suspected of infected hematometra.” 4. Consideration of tubo-o 
--------------------------------------------------------------------------------
PRED: A 75-year-old Caucasian woman was admitted with a prosthetic hip infection 10 days after revision

In [ ]:
SAVE_DIR = "medprompt_t5_small_final"
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

# make a zip to download from Colab
!zip -qr medprompt_t5_small_final.zip {SAVE_DIR}
print("Zipped -> medprompt_t5_small_final.zip")


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3909: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Zipped -> medprompt_t5_small_final.zip


In [ ]:
!ls -lh


total 110M
drwxr-xr-x 2 root root 4.0K Aug 11 08:28 medprompt-t5-small
drwxr-xr-x 2 root root 4.0K Aug 11 09:27 medprompt_t5_small_final
-rw-r--r-- 1 root root 110M Aug 11 09:28 medprompt_t5_small_final.zip
drwxr-xr-x 1 root root 4.0K Aug  7 13:42 sample_data
drwxr-xr-x 3 root root 4.0K Aug 11 09:25 wandb


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch, re

MODEL_DIR = "medprompt_t5_small_final"  # your trained folder

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Safety: decoding config
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
if getattr(model.config, "decoder_start_token_id", None) is None:
    model.config.decoder_start_token_id = tokenizer.pad_token_id
if getattr(model.config, "eos_token_id", None) is None:
    model.config.eos_token_id = tokenizer.eos_token_id or tokenizer.pad_token_id

def infer(symptoms, max_new_tokens=192, num_beams=2):
    # Match training style and add an explicit instruction to reinforce format
    prompt = (
        "You are a clinical reasoning model. "
        "Given Symptoms, reply only in the format: "
        "Reasoning: <short reasoning> || Diagnosis: <final diagnosis>.\n"
        f"Symptoms: {symptoms.strip()}"
    )
    enc = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        out_ids = model.generate(
            **enc,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams
        )
    text = tokenizer.decode(out_ids[0], skip_special_tokens=True)

    # Optional parse
    m = re.search(r"Reasoning:\s*(.*?)\s*\|\|\s*Diagnosis:\s*(.*)$", text, re.S)
    return {"raw": text, "reasoning": (m.group(1).strip() if m else ""), "diagnosis": (m.group(2).strip() if m else "")}


In [ ]:
print(infer("45-year-old with persistent cough, fever, night sweats, weight loss over 1 month")["raw"])


45-year-old with persistent cough, fever, night sweats, weight loss over 1 month. Symptoms: 45-year-old with persistent cough, fever, night sweats, weight loss over 1 month. Symptoms: 45-year-old with persistent cough, fever, night sweats, weight loss over 1 month. Symptoms: 45-year-old with persistent cough, fever, night sweats, weight loss over 1 month.


In [ ]:
from datasets import load_dataset
ds = load_dataset("zou-lab/MedCaseReasoning")
case = ds["val"][0]["case_prompt"][:400]  # trim to keep under 512 tokens after tokenization
print(infer(case)["raw"])


a 48-year-old postmenopausal woman (gravida 2, para 2) presented with a 3-month history of brown-yellowish vaginal discharge, fluctuating lower abdominal pain, fever, and progressive abdominal enlargement over 15 years, worsening in the last 5 years. she reported difficulty with defecation and voiding, but removal was aborted due to severe adhesion.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch, re

MODEL_DIR = "medprompt_t5_small_final"   # your trained folder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tok = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR).to(device)

# Safety defaults
if tok.pad_token_id is None:
    tok.pad_token = tok.eos_token
if getattr(model.config, "decoder_start_token_id", None) is None:
    model.config.decoder_start_token_id = tok.pad_token_id
if getattr(model.config, "eos_token_id", None) is None:
    model.config.eos_token_id = tok.eos_token_id or tok.pad_token_id

# Build a "forced" decoder prefix so outputs start with "Reasoning: "
dec_prefix = "Reasoning: "
dec_prefix_ids = tok(dec_prefix, add_special_tokens=False, return_tensors="pt").input_ids.to(device)

# Ban the model from generating "Symptoms" to reduce echoing
bad = tok(["Symptoms", "Symptoms:"], add_special_tokens=False).input_ids  # list of token-id lists

def infer(symptoms: str):
    # 1) sanitize: keep exactly one "Symptoms:" and strip duplicates
    s = symptoms.strip()
    if not s.lower().startswith("symptoms:"):
        s = "Symptoms: " + s

    enc = tok(s, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        out = model.generate(
            **enc,
            decoder_input_ids=dec_prefix_ids,     # force start with "Reasoning: "
            max_new_tokens=192,
            num_beams=4,
            no_repeat_ngram_size=4,               # cut repeats
            repetition_penalty=1.15,              # punish echoes
            bad_words_ids=bad,                    # prevent "Symptoms" in output
            pad_token_id=tok.pad_token_id,
            eos_token_id=model.config.eos_token_id,
        )

    text = tok.decode(out[0], skip_special_tokens=True)

    # Try to ensure " || Diagnosis: " is present; if not, nudge a second pass
    if "Diagnosis:" not in text:
        text = text.rstrip() + " || Diagnosis: "  # create anchor and let a tiny continuation fill it
        enc2 = tok(s, return_tensors="pt", truncation=True, max_length=512).to(device)
        dec2 = tok(text, add_special_tokens=False, return_tensors="pt").input_ids.to(device)
        with torch.no_grad():
            out2 = model.generate(
                **enc2,
                decoder_input_ids=dec2,
                max_new_tokens=32,
                num_beams=4,
                no_repeat_ngram_size=4,
                repetition_penalty=1.15,
                bad_words_ids=bad,
                pad_token_id=tok.pad_token_id,
                eos_token_id=model.config.eos_token_id,
            )
        text = tok.decode(out2[0], skip_special_tokens=True)

    # Parse nicely
    m = re.search(r"Reasoning:\s*(.*?)\s*\|\|\s*Diagnosis:\s*(.*)$", text, re.S)
    reasoning = m.group(1).strip() if m else ""
    diagnosis = m.group(2).strip() if m else ""
    return {"raw": text, "reasoning": reasoning, "diagnosis": diagnosis}

tests = [
    "High fever, stiff neck, photophobia, severe headache",
    "Chest pain radiating to left arm, sweating, shortness of breath",
    "Lower right abdominal pain, fever, nausea, rebound tenderness"
]
for t in tests:
    r = infer(t)
    print("\nSymptoms:", t)
    print("OUTPUT  :", r["raw"])
    if r["diagnosis"]:
        print("DX      :", r["diagnosis"])


Symptoms: High fever, stiff neck, photophobia, severe headache
OUTPUT  : Reasoning: High fever, stiff neck, photophobia, severe headaches. || Diagnosis: Haute fièvre, hygienophobie, photophobie, sever headaches.
DX      : Haute fièvre, hygienophobie, photophobie, sever headaches.

Symptoms: Chest pain radiating to left arm, sweating, shortness of breath
OUTPUT  : Reasoning: Chest pain radiating to left arm, sweating, shortness of breath, sweating. || Diagnosis: Pain radiating to right arm radiating, sweating and short breath.
DX      : Pain radiating to right arm radiating, sweating and short breath.

Symptoms: Lower right abdominal pain, fever, nausea, rebound tenderness
OUTPUT  : Reasoning: Lower right abdominal pain, fever, nausea, rebound tenderness. || Diagnosis: Low right abdominal pain and fever, nausea and nausea.
DX      : Low right abdominal pain and fever, nausea and nausea.


In [ ]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np

raw = load_dataset("zou-lab/MedCaseReasoning")

train_cases = [f"Symptoms: {x.strip()}" for x in raw["train"]["case_prompt"]]
train_dx = list(raw["train"]["final_diagnosis"])   # <-- make it a Python list

tfidf = TfidfVectorizer(min_df=2, max_df=0.95, ngram_range=(1,2))
X = tfidf.fit_transform(train_cases)

knn = NearestNeighbors(n_neighbors=3, metric="cosine", algorithm="brute")
knn.fit(X)

def retrieve_dx(symptoms: str):
    q = "Symptoms: " + symptoms.strip()
    qv = tfidf.transform([q])
    dists, idxs = knn.kneighbors(qv, n_neighbors=3)
    idxs = [int(i) for i in idxs[0]]               # <-- cast to Python int
    cands = [train_dx[i] for i in idxs]
    best = max(set(cands), key=cands.count)        # majority vote
    return best, list(zip(cands, np.round(dists[0], 3)))


In [ ]:
tests = [
    "High fever, stiff neck, photophobia, severe headache",
    "Chest pain radiating to left arm, sweating, shortness of breath",
    "Lower right abdominal pain, fever, nausea, rebound tenderness",
]
for t in tests:
    best, neigh = retrieve_dx(t)
    print("\nSymptoms:", t)
    print("Pred DX :", best)
    print("Neighbors (dx, dist):", neigh)



Symptoms: High fever, stiff neck, photophobia, severe headache
Pred DX : Chordoma
Neighbors (dx, dist): [('Chordoma', np.float64(0.88)), ('Q fever', np.float64(0.902)), ('tetanus', np.float64(0.914))]

Symptoms: Chest pain radiating to left arm, sweating, shortness of breath
Pred DX : aortic dissection
Neighbors (dx, dist): [('Upper extremity deep vein thrombosis', np.float64(0.846)), ('aortic dissection', np.float64(0.851)), ('Intramedullary arteriovenous malformation', np.float64(0.856))]

Symptoms: Lower right abdominal pain, fever, nausea, rebound tenderness
Pred DX : Retroperitoneal lipoma
Neighbors (dx, dist): [('Osteosarcoma', np.float64(0.836)), ('Retroperitoneal lipoma', np.float64(0.863)), ('appendiceal diverticulitis', np.float64(0.878))]


In [ ]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, top_k_accuracy_score
import numpy as np

raw = load_dataset("zou-lab/MedCaseReasoning")

X_train_text = ["Symptoms: " + s.strip() for s in raw["train"]["case_prompt"]]
y_train_text = list(raw["train"]["final_diagnosis"])

X_val_text = ["Symptoms: " + s.strip() for s in raw["val"]["case_prompt"]]
y_val_text = list(raw["val"]["final_diagnosis"])

# 1) Fit LabelEncoder on the union of labels across splits
all_labels = y_train_text + y_val_text + list(raw["test"]["final_diagnosis"])
le = LabelEncoder().fit(all_labels)

y_train = le.transform(y_train_text)
y_val   = le.transform(y_val_text)

# 2) TF-IDF
tfidf = TfidfVectorizer(min_df=2, max_df=0.95, ngram_range=(1,2),
                        sublinear_tf=True, strip_accents="unicode")
X_train = tfidf.fit_transform(X_train_text)
X_val   = tfidf.transform(X_val_text)

# 3) Train classifier (remove n_jobs if your sklearn is older)
clf = LogisticRegression(max_iter=2000, C=2.0, solver="lbfgs", multi_class="auto")
clf.fit(X_train, y_train)

# 4) Eval
val_pred = clf.predict(X_val)
val_prob = clf.predict_proba(X_val)
top1 = accuracy_score(y_val, val_pred)
top3 = top_k_accuracy_score(y_val, val_prob, k=3)
print(f"Validation Top-1 acc: {top1:.3f}, Top-3 acc: {top3:.3f}")

# 5) Inference helper
def predict_dx(symptoms: str, topk: int = 3):
    qv = tfidf.transform(["Symptoms: " + symptoms.strip()])
    prob = clf.predict_proba(qv)[0]
    top_idx = np.argsort(prob)[::-1][:topk]
    return [(le.inverse_transform([i])[0], float(prob[i])) for i in top_idx]

# Quick test
tests = [
    "High fever, stiff neck, photophobia, severe headache",
    "Chest pain radiating to left arm, sweating, shortness of breath",
    "Lower right abdominal pain, fever, nausea, rebound tenderness",
]
for t in tests:
    print("\nSymptoms:", t)
    print("Top preds:", predict_dx(t, topk=5))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
